In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)
from vessel_manoeuvring_models.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

In [ ]:
from wPCC_pipeline.pipelines.prediction.nodes import simulation_accuracy
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter
from vessel_manoeuvring_models.extended_kalman_vmm import ExtendedKalman

In [ ]:
from vessel_manoeuvring_models.models.vmm import Simulator
from vessel_manoeuvring_models.substitute_dynamic_symbols import run

In [ ]:
id = 22774
df_smooth = catalog.load(f"{ id }.data_ek_smooth")
df_data = catalog.load(f"{ id }.data")
df_data['U'] = np.sqrt(df_data['u']**2 + df_data['v']**2)
df_smooth['U'] = np.sqrt(df_smooth['u']**2 + df_smooth['v']**2)

ship_data = catalog.load("ship_data")

In [ ]:
from wPCC_pipeline.pipelines.brix.nodes import initial_parameters
from wPCC_pipeline.pipelines.vessel_manoeuvring_models.nodes import martins_model, vmm_martins_simple_model, vmm_abkowitz_model

In [ ]:
#vmm = martins_model()
#model = catalog.load(f"vmm_martin.motion_regression.joined.model")

vmm = vmm_martins_simple_model()
model = catalog.load(f"vmm_martins_simple.motion_regression.joined.model")

#vmm = vmm_abkowitz_model()
#model = catalog.load(f"vmm_abkowitz.motion_regression.joined.model")


result = model.simulate(df_smooth)

In [ ]:
parameters = model.parameters.copy()
#parameters['Xudot'] = 0
#parameters['Yvdot'] = 0
#parameters['Yrdot'] = 0
#
#parameters['Nvdot'] = 0
#parameters['Nrdot'] = 0



In [ ]:
model2 = Simulator(X_eq=vmm.X_eq, Y_eq=vmm.Y_eq, N_eq=vmm.N_eq)
model2.define_quasi_static_forces(X_qs_eq=vmm.X_qs_eq, Y_qs_eq=vmm.Y_qs_eq, N_qs_eq=vmm.N_qs_eq)

In [ ]:
ek = ExtendedKalman(vmm=vmm, parameters=parameters, ship_parameters=ship_data)

In [ ]:
result_ek = ek.simulate(data=df_smooth, input_columns=['delta','thrust'], solver='Radau')
#result_ek = ek.simulate(data=df_smooth, input_columns=['delta','thrust'], solver='euler')

In [ ]:
result2 = model2.simulate(df_=df_smooth, parameters=parameters, ship_parameters=ship_data, 
            primed_parameters=True, prime_system=model.prime_system) 

In [ ]:
dataframes = {
#'model test' : df_smooth,
#'simulate runge kutta (result)' : result.result,
'simulate runge kutta (result2)' : result2.result,
'ek sim runge kutta' : result_ek,
}

styles = {

    'model test' : {'style':'-',},
    'simulate runge kutta (result)' : {'style':'-',},
    'simulate runge kutta (result2)' :{'style':':',},
    'ek sim runge kutta' : {'style':'--',},
    

}

fig,ax=plt.subplots()
track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B'], ax=ax, plot_boats=False, styles=styles);

plot(dataframes, keys=result.result.columns, fig_size=(15, 20));

In [ ]:
x = df_smooth[['x0','y0','psi','u','v','r']].values.T
input = df_smooth[['delta','thrust']].copy()
dx = ek.lambda_f(x=x, input = input)
df_acc = pd.DataFrame(index=df_smooth.index, columns=['u1d','v1d','r1d'], data=dx[3:,:].T)

In [ ]:
for key in ['u1d','v1d','r1d']:
    fig,ax=plt.subplots()
    df_acc.plot(y=key, style='r', ax=ax, label='ek')
    result2.result.plot(y=key, style=['b-'], ax=ax, alpha=0.5, label='sim')
    ax.set_ylabel(key)

In [ ]:
df_smooth.plot(y='r')

In [ ]:
df_smooth.plot(y='v')

In [ ]:
df_smooth.plot(y='delta')

In [ ]:
ship_data_prime = model2.prime_system.prime(ship_data)

df_smooth_prime = model2.prime_system.prime(df_smooth[['x0','y0','psi','u','v','r','thrust','delta']], U=df_smooth['U'])
u1d_, v1d_, r1d_ = run(function=model2.acceleration_lambda, **ship_data_prime, **parameters, **df_smooth_prime)

df_acc_sim_prime = pd.DataFrame(index=df_smooth.index)
df_acc_sim_prime['u1d'] = u1d_.flatten()
df_acc_sim_prime['v1d'] = v1d_.flatten()
df_acc_sim_prime['r1d'] = r1d_.flatten()
df_acc_sim = model2.prime_system.unprime(df_acc_sim_prime, U=df_smooth['U'])


In [ ]:
for key in ['u1d','v1d','r1d']:
    fig,ax=plt.subplots()
    df_acc.plot(y=key, style='r', ax=ax, label='ek', lw=0.25)
    df_acc_sim.plot(y=key, style=['b-'], ax=ax, alpha=1, lw=0.25, label='sim')
    ax.set_ylabel(key)

In [ ]:
df_acc/df_acc_sim

In [ ]:
model2.acceleartion_eq

In [ ]:
#A, b = sp.linear_eq_to_matrix([vmm.X_eq, Y_eq, N_eq], [u1d, v1d, r1d])

In [ ]:
parameters

In [ ]:
A, b = sp.linear_eq_to_matrix([vmm.X_eq, vmm.Y_eq, vmm.N_eq], [u1d, v1d, r1d])

In [ ]:
A

In [ ]:
b

In [ ]:
from vessel_manoeuvring_models import prime_system

u_prime, v_prime = sp.symbols("u' v'")


subs_prime = [
            (m, m / prime_system.df_prime.mass.denominator),
            (I_z, I_z / prime_system.df_prime.inertia_moment.denominator),
            (x_G, x_G / prime_system.df_prime.length.denominator),
            
            (u_prime, u / sp.sqrt(u ** 2 + v ** 2)),
            (v_prime, v / sp.sqrt(u ** 2 + v ** 2)),
            
            (r, r / (sp.sqrt(u ** 2 + v ** 2) / L)),
            (thrust, thrust / (sp.Rational(1,2) * rho * (u ** 2 + v ** 2) * L ** 2)),
        ]

In [ ]:
u / sp.sqrt(u**2 + v**2)

In [ ]:
A_SI = A.subs(subs_prime)
b_SI = b.subs(subs_prime)

In [ ]:
A_SI

In [ ]:
b_SI

In [ ]:
u**2+v**2

In [ ]:
A_SI.inv() * b_SI

In [ ]:
sp.Matrix(
                [
                    (u ** 2 + v ** 2) / L,
                    (u ** 2 + v ** 2) / L,
                    (u ** 2 + v ** 2) / (L ** 2),
                ])

In [ ]:
sympy.matrices.dense.matrix_multiply_elementwise(
            A_SI.inv() * b_SI,  # (Slow...)
            sp.Matrix(
                [
                    (u ** 2 + v ** 2) / L,
                    (u ** 2 + v ** 2) / L,
                    (u ** 2 + v ** 2) / (L ** 2),
                ]
            ),
        )